# Automatic Drum Transcription /MIR Project – Teresa Pelinski Ramos

This notebook compares a single task and a multi-task model for the purpose of automatic drum transcription. It uses conda environment.
* The models are already trained


In [1]:
import os
import sys

import pickle
import numpy as np
import glob

import warnings # ignore tensorflow warinings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

from keras import backend as K
import soundfile as psf

sys.path.append('models')
from large_vocab_adt_dafx2018.transcribe import transcribe, PERC_VOICE_SET
import large_vocab_adt_dafx2018.model as model

Using TensorFlow backend.


## Download Groove MIDI Dataset
The dataset we will use for evaluating the CRNN –MT and –ST models is the [Groove MIDI Dataset (GMD) ](https://magenta.tensorflow.org/datasets/groove). This dataset provides 13.6 hours of synthesized outputs, 1,150 MIDI files, and over 22,000 measures of expressive drumming. Most of the performances have a 4/4 time signature and more than the 80% of the dataset comes from hired professionals. In this section the dataset is donwloaded and some stats are shown.

In [2]:
import requests, zipfile, io

# download dataset if not downloaded yet
dataset_path = 'groove/'

# if dir exists and is empty, remove
if os.path.exists(dataset_path) and os.listdir(dataset_path)==0:
    os.removedirs(dataset_path)

# if dir does not exist, download dataset
if not os.path.exists(dataset_path):
    r = requests.get(zip_file_url)
    z = zipfile.ZipFile(io.BytesIO(r.content))
    z.extractall("./")

The dataset comes with an info csv file that contains information for each performance. The last column indicates the dataset split. However, since the CRNN models were trained on different datasets than the GMD, we will not take the split into account when evaluating the models. 

There is also a `beat_type` field that differentiates between `beat` and `fill` performances. `fill` corresponds to short performances of just a couple of seconds while `beat` refers to longer performances of a couple of minutes. 

In [4]:
import pandas as pd

# load info from dataset
magenta_ds = pd.read_csv('groove/info.csv')

# remove rows without wav file
magenta_ds.dropna(axis=0, how='any', thresh=None, subset=None, inplace=True) 

magenta_ds

,drummer,session,id,style,bpm,beat_type,time_signature,midi_filename,audio_filename,duration,split
0,drummer1,drummer1/eval_session,drummer1/eval_session/1,funk/groove1,138,beat,4-4,drummer1/eval_session/1_funk-groove1_138_beat_...,drummer1/eval_session/1_funk-groove1_138_beat_...,27.872308,test
1,drummer1,drummer1/eval_session,drummer1/eval_session/10,soul/groove10,102,beat,4-4,drummer1/eval_session/10_soul-groove10_102_bea...,drummer1/eval_session/10_soul-groove10_102_bea...,37.691158,test
2,drummer1,drummer1/eval_session,drummer1/eval_session/2,funk/groove2,105,beat,4-4,drummer1/eval_session/2_funk-groove2_105_beat_...,drummer1/eval_session/2_funk-groove2_105_beat_...,36.351218,test
3,drummer1,drummer1/eval_session,drummer1/eval_session/3,soul/groove3,86,beat,4-4,drummer1/eval_session/3_soul-groove3_86_beat_4...,drummer1/eval_session/3_soul-groove3_86_beat_4...,44.716543,test
4,drummer1,drummer1/eval_session,drummer1/eval_session/4,soul/groove4,80,beat,4-4,drummer1/eval_session/4_soul-groove4_80_beat_4...,drummer1/eval_session/4_soul-groove4_80_beat_4...,47.987500,test
5,drummer1,drummer1/eval_session,drummer1/eval_session/5,funk/groove5,84,beat,4-4,drummer1/eval_session/5_funk-groove5_84_beat_4...,drummer1/eval_session/5_funk-groove5_84_beat_4...,45.687518,test
6,drummer1,drummer1/eval_session,drummer1/eval_session/6,hiphop/groove6,87,beat,4-4,drummer1/eval_session/6_hiphop-groove6_87_beat...,drummer1/eval_session/6_hiphop-groove6_87_beat...,44.119242,test
7,drummer1,drummer1/eval_session,drummer1/eval_session/7,pop/groove7,138,beat,4-4,drummer1/eval_session/7_pop-groove7_138_beat_4...,drummer1/eval_session/7_pop-groove7_138_beat_4...,27.706547,test
8,drummer1,drummer1/eval_session,drummer1/eval_session/8,rock/groove8,65,beat,4-4,drummer1/eval_session/8_rock-groove8_65_beat_4...,drummer1/eval_session/8_rock-groove8_65_beat_4...,59.067313,test
9,drummer1,drummer1/eval_session,drummer1/eval_session/9,soul/groove9,105,beat,4-4,drummer1/eval_session/9_soul-groove9_105_beat_...,drummer1/eval_session/9_soul-groove9_105_beat_...,36.540504,test
